In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import snowflake.connector
import warnings
from redmail import gmail
from simple_colors import *
import datetime as dt
import requests
import json
import math
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, Button, Layout, HBox, VBox, Box
import ipywidgets as widgets
os.chdir('/Users/jessherbert/Desktop/Critical')
# Connect to Snowflake Databases
conn_recurly=snowflake.connector.connect(user='JESSHERBERT',password='SpartanLife308',account='soa50355',warehouse='COMPUTE_WH',database='DAILYWIRE_RECURLY',schema='CLASSIC')

In [2]:
#################################################################################################################

In [3]:
added=conn_recurly.cursor().execute("""

SELECT DATE(DATEADD(HOURS,-5,ACTIVATED_AT)) DATE, 
       (CASE WHEN PLAN_CODE = 'reader' THEN 'Readers Pass'
             WHEN PLAN_CODE = 'insider' THEN 'Insider Monthly'
             WHEN PLAN_CODE = 'insider_plus' THEN 'Insider Annual'
             WHEN PLAN_CODE = 'all_access' THEN 'All Access'
        END) PLAN, COUNT(*) GAINED_SUBS,
        RATIO_TO_REPORT(GAINED_SUBS) OVER(PARTITION BY DATE) PCT
FROM SUBSCRIPTIONS
WHERE DATE < CURRENT_DATE()
AND PLAN_CODE IN ('reader','insider','insider_plus','all_access')
GROUP BY DATE, PLAN
ORDER BY DATE, (CASE WHEN PLAN = 'Readers Pass' THEN 1
                     WHEN PLAN = 'Insider Monthly' THEN 2
                     WHEN PLAN = 'Insider Annual' THEN 3
                     WHEN PLAN = 'All Access' THEN 4
                     ELSE 5
                END)

""").fetch_pandas_all()

churned=conn_recurly.cursor().execute("""

SELECT DATE(DATEADD(HOURS,-5,EXPIRES_AT)) DATE,
       (CASE WHEN PLAN_CODE = 'reader' THEN 'Readers Pass'
             WHEN PLAN_CODE = 'insider' THEN 'Insider Monthly'
             WHEN PLAN_CODE = 'insider_plus' THEN 'Insider Annual'
             WHEN PLAN_CODE = 'all_access' THEN 'All Access'
        END) PLAN, COUNT(*) LOST_SUBS,
        RATIO_TO_REPORT(LOST_SUBS) OVER(PARTITION BY DATE) PCT
FROM CHURNED_SUBSCRIPTIONS
WHERE DATE < CURRENT_DATE()
AND PLAN_CODE IN ('reader','insider','insider_plus','all_access')
GROUP BY DATE, PLAN
ORDER BY DATE, (CASE WHEN PLAN = 'Readers Pass' THEN 1
                     WHEN PLAN = 'Insider Monthly' THEN 2
                     WHEN PLAN = 'Insider Annual' THEN 3
                     WHEN PLAN = 'All Access' THEN 4
                     ELSE 5
                END)
""").fetch_pandas_all()

net=added.merge(churned,on=['DATE','PLAN'])
net['NET_SUBS'] = net['GAINED_SUBS'] - net['LOST_SUBS']
net=net[['DATE','PLAN','NET_SUBS']]


cancellations=conn_recurly.cursor().execute("""

SELECT DATE(DATEADD(HOURS,-5,CANCELED_AT)) DATE,
       (CASE WHEN PLAN_CODE = 'reader' THEN 'Readers Pass'
             WHEN PLAN_CODE = 'insider' THEN 'Insider Monthly'
             WHEN PLAN_CODE = 'insider_plus' THEN 'Insider Annual'
             WHEN PLAN_CODE = 'all_access' THEN 'All Access'
        END) PLAN, COUNT(*) CANCELS,
        RATIO_TO_REPORT(CANCELS) OVER(PARTITION BY DATE) PCT
FROM SUBSCRIPTIONS
WHERE DATE < CURRENT_DATE()
AND PLAN_CODE IN ('reader','insider','insider_plus','all_access')
GROUP BY DATE, PLAN
ORDER BY DATE, (CASE WHEN PLAN = 'Readers Pass' THEN 1
                     WHEN PLAN = 'Insider Monthly' THEN 2
                     WHEN PLAN = 'Insider Annual' THEN 3
                     WHEN PLAN = 'All Access' THEN 4
                     ELSE 5
                END)

""").fetch_pandas_all()

In [4]:
#items = [Button(description=p) for p in planSelection.options]
#HBox(items)

#items2 = [Button(description = m) for m in metricSelection.options]
#VBox([i for i in items2])

#HBox([VBox(items), VBox(items2)])

#VBox([HBox(items), HBox(items2)])

In [5]:
##################################################################################################################
##################################################################################################################
############################################### DEFINE WIDGETS ###################################################
##################################################################################################################
##################################################################################################################

output = widgets.Output()
output2 = widgets.Output()
plotOutput = widgets.Output()
plotOutput2 = widgets.Output()

# SELECT PLAN
planSelection = widgets.ToggleButtons(
    options=['All','Readers Pass', 'Insider Monthly', 'Insider Annual','All Access'],
    description='Plan:',disabled=False,button_style='',layout = Layout(width='100%',height = '80px'))

# SELECT METRIC
metricSelection = widgets.Dropdown(options=['Added','Churned','Net','Cancellations',
                                           'Distribution - Adds','Distribution - Churned',
                                           'Distribution - Cancellations'])
# SELECT LAST X DAYS
lastSelection = widgets.BoundedIntText(
    value=0,
    min=0,
    max=((dt.date.today()-dt.timedelta(days=1))-min(added.DATE.min(),churned.DATE.min(),
                                                    cancellations.DATE.min())).days,
    step=1, description='Last X Days:', disabled=False)

# SELECT START DATE
dateSelection1 = widgets.DatePicker(description = 'From:', disabled = False)

# SELECT END DATE
dateSelection2 = widgets.DatePicker(description = 'To:', disabled = False)


##################################################################################################################
##################################################################################################################
########################################## DEFINE FILTERING FUNCTION #############################################
##################################################################################################################
##################################################################################################################

def filtering(plan, metric, last, start, end):
    
### CLEAR OUTPUT; OVERALL SETTINGS
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)


    output.clear_output()
    output2.clear_output()
    plotOutput.clear_output()
    plotOutput2.clear_output()
    
    planList = ['Readers Pass', 'Insider Monthly','Insider Annual', 'All Access']
    planList2 = planList+['All']
    colorList = ['#2676a5','#f1dc5b','#723180','#2da07b']
    colorList2 = colorList+['dimgray']
    ticks = []
    tickLabels = []
    pieLabels = []
    c = []
    if plan == 'Readers Pass':
        colorCode = '#2676a5'
    elif plan == 'Insider Monthly':
        colorCode = '#f1dc5b'
    elif plan == 'Insider Annual':
        colorCode = '#723180'
    elif plan == 'All Access':
        colorCode = '#2da07b'
    else:
        colorCode = 'gray'
    
### SET START & END DATES FOR DEFAULT LAST 0 DAYS
    if last == 0:
        if (start is None and end is None):
            date1 = dt.date.today() - dt.timedelta(days = 31)
            date2 = dt.date.today() - dt.timedelta(days = 1)
            
        elif (start is not None and end is None):
            if start < dt.date.today():
                date1 = start
                date2 = dt.date.today() - dt.timedelta(days = 1)
            else:
                date1 = dt.date.today() - dt.timedelta(days = 1)
                date2 = dt.date.today() - dt.timedelta(days = 1)
                
        elif (start is None and end is not None):
            if end < dt.date.today():
                date1 = end - dt.timedelta(days = 31)
                date2 = end
            else:
                date1 = dt.date.today() - dt.timedelta(days=31)
                date2 = dt.date.today() - dt.timedelta(days=1)
        
        elif (start is not None and end is not None):
            if (start < dt.date.today() and end < dt.date.today()):
                if end < start:
                    date1 = start
                    date2 = start
                else:
                    date1 = start
                    date2 = end
                
            else:
                date1 = start
                date2 = dt.date.today() - dt.timedelta(days=1)
                    

### WHEN LAST X DAYS CHOSEN, USE THIS FOR START & END DATES
    else:
        date1 = dt.date.today() - dt.timedelta(days = last)
        date2 = dt.date.today() - dt.timedelta(days = 1)
        

##################################################################################################################
##################################################################################################################
###################################### LIMIT DATA BASED ON METRIC SELECTION ######################################
##################################################################################################################
##################################################################################################################
    
##################################################################################################################
############################################## GROSS ADDS ########################################################
##################################################################################################################
    if (metric == 'Added'):
        if plan == 'All':
            data = added[(added.DATE >= date1) & (added.DATE <= date2)].groupby('DATE').sum().reset_index()
            data2 = added[(added.DATE >= date1) &
                          (added.DATE <= date2)].sort_values(by=['PLAN','DATE'])
            shortDates=[]    
            for i in range(len(data2)):
                shortDates.append(str(int(str(data2.DATE.values[i])[5:7]))+'/'+str(int(str(data2.DATE.values[i])[8:])))
            data2['SHORT'] = shortDates
            
            with output2:
                warnings.filterwarnings("ignore")
                tempTable = data2.set_index('PLAN').sort_values(by=['PLAN',
                            'DATE'],ascending=[False,True]).reset_index()
                dataTable = pd.DataFrame([])
                for i in planList:
                    subset = tempTable[tempTable.PLAN == i]
                    subset = subset[['DATE',
                            'GAINED_SUBS']].set_index('DATE').rename(columns={'GAINED_SUBS':''}).transpose()
                    subset.insert(0, 'PLAN',i)
                    dataTable = dataTable.append(subset)
                display(dataTable)
                
            with plotOutput2:
                    
                plt.figure(figsize=(20,6))
                
                for j in range(len(planList)):
                    planName = planList[j]
                    subset = data2[data2.PLAN == planName]
                    avg = "{:,}".format(round(subset.GAINED_SUBS.values.mean()))
                    total = "{:,}".format(round(subset.GAINED_SUBS.values.sum()))
                    
                    if last == 0:
                        if (date1 == date2):
                            plt.scatter(range(len(subset)),
                                   subset.GAINED_SUBS.values,
                                   label = planName+': '+total+' ('+avg+'/day)',
                                   color = colorList[j],
                                   edgecolor = 'k')
                        else:
                            plt.plot(range(len(subset)),
                                subset.GAINED_SUBS.values,
                                label = planName+': '+total+' ('+avg+'/day)',
                                color = colorList[j])

                    elif last >= 3:
                        plt.plot(range(len(subset)),
                                subset.GAINED_SUBS.values,
                                label = planName+': '+total+' ('+avg+'/day)',
                                color = colorList[j])
                    else:
                        plt.scatter(range(len(subset)),
                                   subset.GAINED_SUBS.values,
                                   label = planName+': '+total+' ('+avg+'/day)',
                                   color = colorList[j],
                                   edgecolor = 'k')
                    if len(data2)/4 < 45:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index())),
                                  labels = subset.SHORT.values)
                    elif len(data2)/4 <= 300:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 10),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::10])
                    elif len(data2)/4 <= 600:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 20),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::20])
                    elif len(data2)/4 <= 1500:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 50),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::50])
                    else:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 100),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::100])
                plt.title(metric+' (By Day)', fontsize = 12, fontweight = 'bold')
                plt.ylim(bottom = 0, top = data2.GAINED_SUBS.max()*1.1)
                plt.axhline(y=0, color='k')
                plt.legend(bbox_to_anchor=(1,1))
                plt.show()
            
        else:
            data = added[(added.PLAN == plan) &
                         (added.DATE >= date1) & (added.DATE <= date2)]
        shortDates=[]    
        for i in range(len(data)):
            shortDates.append(str(int(str(data.DATE.values[i])[5:7]))+'/'+str(int(str(data.DATE.values[i])[8:])))
        data['SHORT'] = shortDates
        outcomes = data.GAINED_SUBS.values
        
##################################################################################################################
############################################ DISTRIBUTION OF ADDS ################################################
##################################################################################################################
    elif (metric == 'Distribution - Adds'):      
# ALL PLANS
        if plan == 'All':
        
            ### AGGREGATE FOR DATE RANGE
            data = added[(added.DATE >= date1) & (added.DATE <= date2)].groupby('PLAN').sum().reset_index()
            data['PCT'] = data['GAINED_SUBS'] / data.GAINED_SUBS.sum()
            
            ### BY DAY BY PLAN
            data2 = added[(added.DATE >= date1) &
                          (added.DATE <= date2)].sort_values(by=['PLAN','DATE'])
            shortDates=[]    
            for i in range(len(data2)):
                shortDates.append(str(int(str(data2.DATE.values[i])[5:7]))+'/'+str(int(str(data2.DATE.values[i])[8:])))
            data2['SHORT'] = shortDates
            
            with output2:
                warnings.filterwarnings("ignore")
                tempTable = data2.set_index('PLAN').sort_values(by=['PLAN',
                            'DATE'],ascending=[False,True]).reset_index()
                dataTable = pd.DataFrame([])
                
                for i in planList:
                    subset = tempTable[tempTable.PLAN == i]
                    strPct = []
                    for j in range(len(subset)):
                        pct = subset.PCT.values[j]
                        strPct.append(str(int(round(100*pct)))+'%')
                    subset['PCT'] = strPct
                    subset = subset[['DATE',
                            'PCT']].set_index('DATE').rename(columns={'PCT':''}).transpose()
                    subset.insert(0, 'PLAN',i)
                    dataTable = dataTable.append(subset)
                display(dataTable)
                
            with plotOutput2:
                plt.figure (figsize = (20,6))

                ### 45 days or greater
                if len(data2)/4 >= 45:
                    for j in range(len(planList)):
                        planName = planList[j]
                        subset = data2[data2.PLAN == planName]
                        avg = subset.PCT.values.mean()
                        plt.plot(range(len(subset)),
                                subset.PCT.values,
                                 label = planName+' ~ '+str(round(100*avg))+'%',
                                color = colorList[j])
                        if len(data2)/4 <= 300:
                            plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                    step = 10),
                                      labels = data2.groupby(['DATE',
                                                             'SHORT']).sum().reset_index().SHORT.values[::10])
                        elif len(data2)/4 <= 600:
                            plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                    step = 20),
                                      labels = data2.groupby(['DATE',
                                                             'SHORT']).sum().reset_index().SHORT.values[::20])
                        elif len(data2)/4 <= 1500:
                            plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                    step = 50),
                                      labels = data2.groupby(['DATE',
                                                             'SHORT']).sum().reset_index().SHORT.values[::50])
                        else:
                            plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                    step = 100),
                                      labels = data2.groupby(['DATE',
                                                             'SHORT']).sum().reset_index().SHORT.values[::100])

                ### LESS THAN 45 DAYS
                else:
                    b = 0
                    for j in range(len(planList)):
                        planName = planList[j]
                        subset = data2[data2.PLAN == planName]
                        avg = subset.PCT.values.mean()
                        plt.bar(range(len(subset)),
                                subset.PCT.values,
                                label = planName+' ~ '+str(round(100*avg))+'%', bottom = b,
                                color = colorList[j], edgecolor = 'k')
                        b = b + subset.PCT.values
                    plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index())),
                              labels = data2.groupby('SHORT').sum().reset_index().SHORT.values,
                              rotation = 45)
                plt.title(metric+' (By Day)', fontsize = 12, fontweight = 'bold')
                plt.ylim(bottom = 0, top = 1)
                plt.yticks(ticks = np.arange(0,1.05,step=.05),
                          labels = ['0%','','10%','','20%','','30%','','40%','','50%','',
                                   '60%','','70%','','80%','','90%','','100%'])
                plt.axhline(y=0, color='k')
                plt.legend(bbox_to_anchor=(1,1))
                plt.show()
                
# SPECIFIC PLAN  
        else:
            data = added[(added.PLAN == plan) &
                         (added.DATE >= date1) & (added.DATE <= date2)]
            shortDates=[]    
            for i in range(len(data)):
                shortDates.append(str(int(str(data.DATE.values[i])[5:7]))+'/'+str(int(str(data.DATE.values[i])[8:])))
            data['SHORT'] = shortDates
        outcomes = data.PCT.values
        
##################################################################################################################
################################################ CHURNED #########################################################
##################################################################################################################
    elif (metric == 'Churned'):
        if plan == 'All':
            data = churned[(churned.DATE >= date1) &
                           (churned.DATE <= date2)].groupby('DATE').sum().reset_index()
            data2 = churned[(churned.DATE >= date1) &
                          (churned.DATE <= date2)].sort_values(by=['PLAN','DATE'])
            shortDates=[]    
            for i in range(len(data2)):
                shortDates.append(str(int(str(data2.DATE.values[i])[5:7]))+'/'+str(int(str(data2.DATE.values[i])[8:])))
            data2['SHORT'] = shortDates
            
            with output2:
                warnings.filterwarnings("ignore")
                tempTable = data2.set_index('PLAN').sort_values(by=['PLAN',
                            'DATE'],ascending=[False,True]).reset_index()
                dataTable = pd.DataFrame([])
                for i in planList:
                    subset = tempTable[tempTable.PLAN == i]
                    subset = subset[['DATE',
                            'LOST_SUBS']].set_index('DATE').rename(columns={'LOST_SUBS':''}).transpose()
                    subset.insert(0, 'PLAN',i)
                    dataTable = dataTable.append(subset)
                display(dataTable)
                
            with plotOutput2:
                plt.figure(figsize=(20,6))
                
                for j in range(len(planList)):
                    planName = planList[j]
                    subset = data2[data2.PLAN == planName]
                    avg = "{:,}".format(round(subset.LOST_SUBS.values.mean()))
                    total = "{:,}".format(round(subset.LOST_SUBS.values.sum()))
                    
                    if last == 0:
                        if (date1 == date2):
                            plt.scatter(range(len(subset)),
                                   subset.LOST_SUBS.values,
                                   label = planName+': '+total+' ('+avg+'/day)',
                                   color = colorList[j],
                                   edgecolor = 'k')
                        else:
                            plt.plot(range(len(subset)),
                                subset.LOST_SUBS.values,
                                label = planName+': '+total+' ('+avg+'/day)',
                                color = colorList[j])
                    
                    elif last >= 3 :
                        plt.plot(range(len(subset)),
                                subset.LOST_SUBS.values,
                                label = planName+': '+total+' ('+avg+'/day)',
                                color = colorList[j])
                    else:
                        plt.scatter(range(len(subset)),
                                   subset.LOST_SUBS.values,
                                   label = planName+': '+total+' ('+avg+'/day)',
                                   color = colorList[j],
                                   edgecolor = 'k')
                    if len(data2)/4 < 45:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index())),
                                  labels = subset.SHORT.values)
                    elif len(data2)/4 <= 300:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 10),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::10])
                    elif len(data2)/4 <= 600:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 20),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::20])
                    elif len(data2)/4 <= 1500:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 50),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::50])
                    else:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 100),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::100])
                plt.title(metric+' (By Day)', fontsize = 12, fontweight = 'bold')
                plt.ylim(bottom = 0, top = data2.LOST_SUBS.max()*1.1)
                plt.axhline(y=0, color='k')
                plt.legend(bbox_to_anchor=(1,1))
                plt.show()
            
        else:
            data = churned[(churned.PLAN == plan) &
                          (churned.DATE >= date1) & (churned.DATE <= date2)]
        shortDates=[]    
        for i in range(len(data)):
            shortDates.append(str(int(str(data.DATE.values[i])[5:7]))+'/'+str(int(str(data.DATE.values[i])[8:])))
        data['SHORT'] = shortDates
        outcomes = data.LOST_SUBS.values
        
##################################################################################################################
########################################### DISTRIBUTION OF CHURNED ##############################################
##################################################################################################################        
    elif (metric == 'Distribution - Churned'):
# ALL PLANS
        if plan == 'All':
        
            ### AGGREGATE FOR DATE RANGE
            data = churned[(churned.DATE >= date1) & (churned.DATE <= date2)].groupby('PLAN').sum().reset_index()
            data['PCT'] = data['LOST_SUBS'] / data.LOST_SUBS.sum()
            
            ### BY DAY BY PLAN
            data2 = churned[(churned.DATE >= date1) &
                          (churned.DATE <= date2)].sort_values(by=['PLAN','DATE'])
            shortDates=[]    
            for i in range(len(data2)):
                shortDates.append(str(int(str(data2.DATE.values[i])[5:7]))+'/'+str(int(str(data2.DATE.values[i])[8:])))
            data2['SHORT'] = shortDates
            
            with output2:
                warnings.filterwarnings("ignore")
                tempTable = data2.set_index('PLAN').sort_values(by=['PLAN',
                            'DATE'],ascending=[False,True]).reset_index()
                dataTable = pd.DataFrame([])
                
                for i in planList:
                    subset = tempTable[tempTable.PLAN == i]
                    strPct = []
                    for j in range(len(subset)):
                        pct = subset.PCT.values[j]
                        strPct.append(str(int(round(100*pct)))+'%')
                    subset['PCT'] = strPct
                    subset = subset[['DATE',
                            'PCT']].set_index('DATE').rename(columns={'PCT':''}).transpose()
                    subset.insert(0, 'PLAN',i)
                    dataTable = dataTable.append(subset)
                display(dataTable)
            
            with plotOutput2:
                plt.figure (figsize = (20,6))

                ### 45 days or greater
                if len(data2)/4 >= 45:
                    for j in range(len(planList)):
                        planName = planList[j]
                        subset = data2[data2.PLAN == planName]
                        avg = subset.PCT.values.mean()
                        plt.plot(range(len(subset)),
                                subset.PCT.values,
                                 label = planName+' ~ '+str(round(100*avg))+'%',
                                color = colorList[j])
                        if len(data2)/4 <= 300:
                            plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                    step = 10),
                                      labels = data2.groupby(['DATE',
                                                             'SHORT']).sum().reset_index().SHORT.values[::10])
                        elif len(data2)/4 <= 600:
                            plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                    step = 20),
                                      labels = data2.groupby(['DATE',
                                                             'SHORT']).sum().reset_index().SHORT.values[::20])
                        elif len(data2)/4 <= 1500:
                            plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                    step = 50),
                                      labels = data2.groupby(['DATE',
                                                             'SHORT']).sum().reset_index().SHORT.values[::50])
                        else:
                            plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                    step = 100),
                                      labels = data2.groupby(['DATE',
                                                             'SHORT']).sum().reset_index().SHORT.values[::100])

                ### LESS THAN 45 DAYS
                else:
                    b = 0
                    for j in range(len(planList)):
                        planName = planList[j]
                        subset = data2[data2.PLAN == planName]
                        avg = subset.PCT.values.mean()
                        plt.bar(range(len(subset)),
                                subset.PCT.values,
                                label = planName+' ~ '+str(round(100*avg))+'%', bottom = b,
                                color = colorList[j], edgecolor = 'k')
                        b = b + subset.PCT.values
                    plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index())),
                              labels = data2.groupby('SHORT').sum().reset_index().SHORT.values,
                              rotation = 45)
                plt.title(metric+' (By Day)', fontsize = 12, fontweight = 'bold')
                plt.ylim(bottom = 0, top = 1)
                plt.yticks(ticks = np.arange(0,1.05,step=.05),
                          labels = ['0%','','10%','','20%','','30%','','40%','','50%','',
                                   '60%','','70%','','80%','','90%','','100%'])
                plt.axhline(y=0, color='k')
                plt.legend(bbox_to_anchor=(1,1))
                plt.show()
            
# SPECIFIC PLAN  
        else:
            data = churned[(churned.PLAN == plan) &
                         (churned.DATE >= date1) & (churned.DATE <= date2)]
            shortDates=[]    
            for i in range(len(data)):
                shortDates.append(str(int(str(data.DATE.values[i])[5:7]))+'/'+str(int(str(data.DATE.values[i])[8:])))
            data['SHORT'] = shortDates
        outcomes = data.PCT.values
        
##################################################################################################################
################################################## NET ###########################################################
##################################################################################################################
    elif (metric == 'Net'):
        if plan == 'All':
            data = net[(net.DATE >= date1) & (net.DATE <= date2)].groupby('DATE').sum().reset_index()
            data2 = net[(net.DATE >= date1) &
                          (net.DATE <= date2)].sort_values(by=['PLAN','DATE'])
            shortDates=[]    
            for i in range(len(data2)):
                shortDates.append(str(int(str(data2.DATE.values[i])[5:7]))+'/'+str(int(str(data2.DATE.values[i])[8:])))
            data2['SHORT'] = shortDates
            
            with output2:
                warnings.filterwarnings("ignore")
                tempTable = data2.set_index('PLAN').sort_values(by=['PLAN',
                            'DATE'],ascending=[False,True]).reset_index()
                dataTable = pd.DataFrame([])
                for i in planList:
                    subset = tempTable[tempTable.PLAN == i]
                    subset = subset[['DATE',
                            'NET_SUBS']].set_index('DATE').rename(columns={'NET_SUBS':''}).transpose()
                    subset.insert(0, 'PLAN',i)
                    dataTable = dataTable.append(subset)
                display(dataTable)
                
            with plotOutput2:
                plt.figure(figsize=(20,6))
                
                for j in range(len(planList)):
                    planName = planList[j]
                    subset = data2[data2.PLAN == planName]
                    avg = "{:,}".format(round(subset.NET_SUBS.values.mean()))
                    total = "{:,}".format(round(subset.NET_SUBS.values.sum()))
                    
                    if last == 0:
                        if (date1 == date2):
                            plt.scatter(range(len(subset)),
                                   subset.NET_SUBS.values,
                                   label = planName+': '+total+' ('+avg+'/day)',
                                   color = colorList[j],
                                   edgecolor = 'k')
                        else:
                            plt.plot(range(len(subset)),
                                subset.NET_SUBS.values,
                                label = planName+': '+total+' ('+avg+'/day)',
                                color = colorList[j])
                    
                    elif last >= 3:
                        plt.plot(range(len(subset)),
                                subset.NET_SUBS.values,
                                label = planName+': '+total+' ('+avg+'/day)',
                                color = colorList[j])
                    else:
                        plt.scatter(range(len(subset)),
                                   subset.NET_SUBS.values,
                                   label = planName+': '+total+' ('+avg+'/day)',
                                   color = colorList[j],
                                   edgecolor = 'k')
                    if len(data2)/4 < 45:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index())),
                                  labels = subset.SHORT.values)
                    elif len(data2)/4 <= 300:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 10),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::10])
                    elif len(data2)/4 <= 600:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 20),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::20])
                    elif len(data2)/4 <= 1500:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 50),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::50])
                    else:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 100),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::100])
                plt.title(metric+' (By Day)', fontsize = 12, fontweight = 'bold')
                plt.ylim(bottom = min(0,data2.NET_SUBS.min()*1.1), top = max(0,data2.NET_SUBS.max()*1.1))
                plt.axhline(y=0, color='k')
                plt.legend(bbox_to_anchor=(1,1))
                plt.show()
        else:
            data = net[(net.PLAN == plan) &
                      (net.DATE >= date1) & (net.DATE <= date2)]
        shortDates=[]    
        for i in range(len(data)):
            shortDates.append(str(int(str(data.DATE.values[i])[5:7]))+'/'+str(int(str(data.DATE.values[i])[8:])))
        data['SHORT'] = shortDates
        outcomes = data.NET_SUBS.values
        
##################################################################################################################
############################################### CANCELLATIONS ####################################################
##################################################################################################################
    elif (metric == 'Cancellations'):
        if plan == 'All':
            data = cancellations[(cancellations.DATE >= date1) &
                                 (cancellations.DATE <= date2)].groupby('DATE').sum().reset_index()
            data2 = cancellations[(cancellations.DATE >= date1) &
                          (cancellations.DATE <= date2)].sort_values(by=['PLAN','DATE'])
            shortDates=[]    
            for i in range(len(data2)):
                shortDates.append(str(int(str(data2.DATE.values[i])[5:7]))+'/'+str(int(str(data2.DATE.values[i])[8:])))
            data2['SHORT'] = shortDates
            
            with output2:
                warnings.filterwarnings("ignore")
                tempTable = data2.set_index('PLAN').sort_values(by=['PLAN',
                            'DATE'],ascending=[False,True]).reset_index()
                dataTable = pd.DataFrame([])
                for i in planList:
                    subset = tempTable[tempTable.PLAN == i]
                    subset = subset[['DATE',
                            'CANCELS']].set_index('DATE').rename(columns={'CANCELS':''}).transpose()
                    subset.insert(0, 'PLAN',i)
                    dataTable = dataTable.append(subset)
                display(dataTable)
                
            with plotOutput2:
                plt.figure(figsize=(20,6))
                
                for j in range(len(planList)):
                    planName = planList[j]
                    subset = data2[data2.PLAN == planName]
                    avg = "{:,}".format(round(subset.CANCELS.values.mean()))
                    total = "{:,}".format(round(subset.CANCELS.values.sum()))
                    
                    if last == 0:
                        if (date1 == date2):
                            plt.scatter(range(len(subset)),
                                   subset.CANCELS.values,
                                   label = planName+': '+total+' ('+avg+'/day)',
                                   color = colorList[j],
                                   edgecolor = 'k')
                        else:
                            plt.plot(range(len(subset)),
                                subset.CANCELS.values,
                                label = planName+': '+total+' ('+avg+'/day)',
                                color = colorList[j])
                    
                    elif last >= 3 :
                        plt.plot(range(len(subset)),
                                 subset.CANCELS.values,
                                 label = planName+': '+total+' ('+avg+'/day)',
                                 color = colorList[j])
                    else:
                        plt.scatter(range(len(subset)),
                                   subset.CANCELS.values,
                                   label = planName+': '+total+' ('+avg+'/day)',
                                   color = colorList[j],
                                   edgecolor = 'k')
                    if len(data2)/4 < 45:
                        plt.xticks(ticks = np.arange(len(subset.groupby('DATE').sum().reset_index())),
                                  labels = subset.SHORT.values)
                    elif len(data2)/4 <= 300:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 10),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::10])
                    elif len(data2)/4 <= 600:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 20),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::20])
                    elif len(data2)/4 <= 1500:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 50),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::50])
                    else:
                        plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                step = 100),
                                    labels = data2.groupby(['DATE',
                                                            'SHORT']).sum().reset_index().SHORT.values[::100])
                plt.title(metric+' (By Day)', fontsize = 12, fontweight = 'bold')
                plt.ylim(bottom = 0, top = data2.CANCELS.max()*1.1)
                plt.axhline(y=0, color='k')
                plt.legend(bbox_to_anchor=(1,1))
                plt.show()
        else:
            data = cancellations[(cancellations.PLAN == plan) &
                                (cancellations.DATE >= date1) & (cancellations.DATE <= date2)]
        shortDates=[]    
        for i in range(len(data)):
            shortDates.append(str(int(str(data.DATE.values[i])[5:7]))+'/'+str(int(str(data.DATE.values[i])[8:])))
        data['SHORT'] = shortDates
        outcomes = data.CANCELS.values
        
##################################################################################################################        
####################################### DISTRIBUTION OF CANCELLATIONS ############################################
##################################################################################################################
    elif (metric == 'Distribution - Cancellations'):
# ALL PLANS
        if plan == 'All':
        
            ### AGGREGATE FOR DATE RANGE
            data = cancellations[(cancellations.DATE >= date1) & (cancellations.DATE <= date2)].groupby('PLAN').sum().reset_index()
            data['PCT'] = data['CANCELS'] / data.CANCELS.sum()
            
            ### BY DAY BY PLAN
            data2 = cancellations[(cancellations.DATE >= date1) &
                          (cancellations.DATE <= date2)].sort_values(by=['PLAN','DATE'])
            shortDates=[]    
            for i in range(len(data2)):
                shortDates.append(str(int(str(data2.DATE.values[i])[5:7]))+'/'+str(int(str(data2.DATE.values[i])[8:])))
            data2['SHORT'] = shortDates
            
            with output2:
                warnings.filterwarnings("ignore")
                tempTable = data2.set_index('PLAN').sort_values(by=['PLAN',
                            'DATE'],ascending=[False,True]).reset_index()
                dataTable = pd.DataFrame([])
                
                for i in planList:
                    subset = tempTable[tempTable.PLAN == i]
                    strPct = []
                    for j in range(len(subset)):
                        pct = subset.PCT.values[j]
                        strPct.append(str(int(round(100*pct)))+'%')
                    subset['PCT'] = strPct
                    subset = subset[['DATE',
                            'PCT']].set_index('DATE').rename(columns={'PCT':''}).transpose()
                    subset.insert(0, 'PLAN',i)
                    dataTable = dataTable.append(subset)
                display(dataTable)
            
            with plotOutput2:
                plt.figure (figsize = (20,6))

                ### 45 days or greater
                if len(data2)/4 >= 45:
                    for j in range(len(planList)):
                        planName = planList[j]
                        subset = data2[data2.PLAN == planName]
                        avg = subset.PCT.values.mean()
                        plt.plot(range(len(subset)),
                                subset.PCT.values,
                                 label = planName+' ~ '+str(round(100*avg))+'%',
                                color = colorList[j])
                        if len(data2)/4 <= 300:
                            plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                    step = 10),
                                      labels = data2.groupby(['DATE',
                                                             'SHORT']).sum().reset_index().SHORT.values[::10])
                        elif len(data2)/4 <= 600:
                            plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                    step = 20),
                                      labels = data2.groupby(['DATE',
                                                             'SHORT']).sum().reset_index().SHORT.values[::20])
                        elif len(data2)/4 <= 1500:
                            plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                    step = 50),
                                      labels = data2.groupby(['DATE',
                                                             'SHORT']).sum().reset_index().SHORT.values[::50])
                        else:
                            plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index()),
                                                    step = 100),
                                      labels = data2.groupby(['DATE',
                                                             'SHORT']).sum().reset_index().SHORT.values[::100])

                ### LESS THAN 45 DAYS
                else:
                    b = 0
                    for j in range(len(planList)):
                        planName = planList[j]
                        subset = data2[data2.PLAN == planName]
                        avg = subset.PCT.values.mean()
                        plt.bar(range(len(subset)),
                                subset.PCT.values,
                                label = planName+' ~ '+str(round(100*avg))+'%', bottom = b,
                                color = colorList[j], edgecolor = 'k')
                        b = b + subset.PCT.values
                    plt.xticks(ticks = np.arange(len(data2.groupby('DATE').sum().reset_index())),
                              labels = data2.groupby('SHORT').sum().reset_index().SHORT.values,
                              rotation = 45)
                plt.title(metric+' (By Day)', fontsize = 12, fontweight = 'bold')
                plt.ylim(bottom = 0, top = 1)
                plt.yticks(ticks = np.arange(0,1.05,step=.05),
                          labels = ['0%','','10%','','20%','','30%','','40%','','50%','',
                                   '60%','','70%','','80%','','90%','','100%'])
                plt.axhline(y=0, color='k')
                plt.legend(bbox_to_anchor=(1,1))
                plt.show()
            
        else:
            data = cancellations[(cancellations.PLAN == plan) &
                         (cancellations.DATE >= date1) & (cancellations.DATE <= date2)]
            shortDates=[]    
            for i in range(len(data)):
                shortDates.append(str(int(str(data.DATE.values[i])[5:7]))+'/'+str(int(str(data.DATE.values[i])[8:])))
            data['SHORT'] = shortDates
        outcomes = data.PCT.values
        
    
##################################################################################################################
##################################################################################################################
############################################## DISPLAY OUTPUTS ###################################################
##################################################################################################################
##################################################################################################################

##################################################################################################################        
########################################## DISPLAY FILTERED DATA #################################################
##################################################################################################################
    with output:
        warnings.filterwarnings("ignore")
        dataTable = data.copy(deep=True).rename(columns={'GAINED_SUBS':'VALUE','LOST_SUBS':'VALUE',
                                                'NET_SUBS':'VALUE','CANCELS':'VALUE'})
        
        if plan == 'All':
            if metric not in ('Distribution - Adds','Distribution - Churned', 'Distribution - Cancellations'):
                dataTable = dataTable[['DATE',
                                       'VALUE']].rename(columns={'VALUE':''}).set_index('DATE').transpose()
            else:
                strPct = []
                for i in range(len(dataTable)):
                    pct = dataTable.PCT.values[i]
                    strPct.append(str(int(round(100*float(pct),0)))+'%')
                dataTable['PCT'] = strPct
                dataTable = dataTable.rename(columns={'VALUE':'',
                            'PCT':''}).set_index('PLAN').sort_index(ascending=False).transpose()
        else:
            
            if metric not in ('Distribution - Adds','Distribution - Churned', 'Distribution - Cancellations'):
                dataTable = dataTable[['DATE',
                                       'VALUE']].rename(columns={'VALUE':''}).set_index('DATE').transpose()
            else:
                dataTable = dataTable[['DATE','PCT']]
                strPct = []
                for i in range(len(dataTable)):
                    pct = dataTable.PCT.values[i]
                    strPct.append(str(int(round(100*float(pct),0)))+'%')
                dataTable['PCT'] = strPct
                dataTable = dataTable.rename(columns={'PCT':''}).set_index('DATE').transpose()
                
        
        display(dataTable)
        
##################################################################################################################        
############################################### DISPLAY PLOTS ####################################################
################################################################################################################## 
    with plotOutput:
        plt.figure (figsize = (20,10))
        
# DISTRIBUTIONS
        if metric in ('Distribution - Adds','Distribution - Churned', 'Distribution - Cancellations'):
            if plan == 'All':
                for i in range(len(data)):
                    planName = data.PLAN.values[i]
                    if planName == 'Readers Pass':
                        planAbrv = 'RP'
                        col = '#2676a5'
                    elif planName == 'Insider Monthly':
                        planAbrv = 'IM'
                        col = '#f1dc5b'
                    elif planName == 'Insider Annual':
                        planAbrv = 'IA'
                        col = '#723180'
                    elif planName == 'All Access':
                        planAbrv = 'AA'
                        col = '#2da07b'
                    pieLabels.append(planAbrv)
                    c.append(col)
                plt.pie(outcomes,labels=pieLabels,colors = c, startangle = 90, autopct = '%1.f%%',
                        wedgeprops = {'edgecolor':'k'},
                        textprops = {'fontweight':'bold','fontsize':'12'})
                
            else:
                if len(data) >= 45:
                    plt.plot(range(len(data)), outcomes,
                            color = colorCode)
                    if len(data) <= 300:
                        plt.xticks(ticks = np.arange(len(data),step = 10),
                                  labels = data.SHORT.values[::10])
                    elif len(data) <= 600:
                        plt.xticks(ticks = np.arange(len(data),step = 20),
                                  labels = data.SHORT.values[::20])
                    elif len(data) <= 1500:
                        plt.xticks(ticks = np.arange(len(data),step = 50),
                                  labels = data.SHORT.values[::50])
                    else:
                        plt.xticks(ticks = np.arange(len(data),step = 100),
                                  labels = data.SHORT.values[::100])
                        
                else:
                    plt.bar(range(len(data)), outcomes,
                           color = colorCode, edgecolor = 'k')
                    plt.xticks(ticks = np.arange(len(data)),
                          labels = shortDates,
                              rotation = 45)
                    for i in range(len(data)):
                        val = outcomes[i]
                        plt.annotate(str(round(100*val))+'%',(i-.1,float(val)*1.025), rotation = 45)
                plt.ylim(bottom = 0, top = 1)
                plt.yticks(ticks = np.arange(0,1.05,step=.05),
                          labels = ['0%','','10%','','20%','','30%','','40%','','50%','',
                                   '60%','','70%','','80%','','90%','','100%'])
                plt.plot(range(len(data)),[outcomes.mean()]*len(data),
                        linestyle=':', color = 'k',
                        label = 'Average: '+str(round(100*outcomes.mean()))+'%')
                plt.legend()

# ADDS, CHURN, NET, CANCELLATIONS
        else:
            if len(data) >= 45:
                plt.plot(range(len(data)), outcomes,
                        color = colorList2[planList2.index(plan)])
                if len(data) <= 300:
                        plt.xticks(ticks = np.arange(len(data),step = 10),
                                  labels = data.SHORT.values[::10])
                elif len(data) <= 600:
                    plt.xticks(ticks = np.arange(len(data),step = 20),
                                labels = data.SHORT.values[::20])
                elif len(data) <= 1500:
                    plt.xticks(ticks = np.arange(len(data),step = 50),
                                labels = data.SHORT.values[::50])
                else:
                    plt.xticks(ticks = np.arange(len(data),step = 100),
                                labels = data.SHORT.values[::100])
            else:
                for i in range(len(data)):
                    value = outcomes[i]
                    if value <= 0 :
                        c = 'crimson'
                        multiplier = 0
                        addition = 5
                    else:
                        c = colorList2[planList2.index(plan)]
                        multiplier = 1.025
                        addition = 0
                    plt.bar(i,value,color = c, edgecolor = 'k')
                    plt.annotate("{:,}".format(value),(i-.1,(value*multiplier)+addition),rotation=45)
                    plt.xticks(ticks = np.arange(len(data)),
                              labels = data.SHORT.values,
                              rotation = 45)
            
            plt.ylim(bottom = min(0, outcomes.min()*1.1),
                top = max(outcomes.max()*1.25,100))
            plt.axhline(y=0, color='k')
            plt.title('Daily Average =  '+str("{:,}".format(round(outcomes.mean())))+
                      '             Total = '+str("{:,}".format(outcomes.sum())))
            plt.plot(range(len(data)), [outcomes.mean()]*len(data), linestyle = ':', color = 'k', label = 'Average')
        plt.suptitle(metric+': '+plan, fontsize = 15, fontweight = 'bold')
        plt.legend(bbox_to_anchor = (1,1))
        plt.show()
            
        
#############################################################################################################
#############################################################################################################
#############################################################################################################

### EVENT HANDLERS
def planHandler(change):
    filtering(change.new, metricSelection.value,
              lastSelection.value, dateSelection1.value, dateSelection2.value)
    
def metricHandler(change):
    filtering(planSelection.value, change.new,
              lastSelection.value, dateSelection1.value, dateSelection2.value)
    
def lastHandler(change):
    filtering(planSelection.value, metricSelection.value,
             change.new, dateSelection1.value, dateSelection2.value)
    
def date1Handler(change):
    filtering(planSelection.value, metricSelection.value,
             lastSelection.value, change.new, dateSelection2.value)
    
def date2Handler(change):
    filtering(planSelection.value, metricSelection.value,
             lastSelection.value, dateSelection1.value, change.new)
    

#############################################################################################################
#############################################################################################################
#############################################################################################################    

    
### OBSERVE OUTPUT
planSelection.observe(planHandler, names = 'value')
metricSelection.observe(metricHandler, names = 'value')
lastSelection.observe(lastHandler, names = 'value')
dateSelection1.observe(date1Handler, names = 'value')
dateSelection2.observe(date2Handler, names = 'value')

In [6]:
display(planSelection,metricSelection, dateSelection1, dateSelection2,lastSelection)
display(plotOutput2)
display(output2)
display(plotOutput)
display(output)

ToggleButtons(description='Plan:', layout=Layout(height='80px', width='100%'), options=('All', 'Readers Pass',…

Dropdown(options=('Added', 'Churned', 'Net', 'Cancellations', 'Distribution - Adds', 'Distribution - Churned',…

DatePicker(value=None, description='From:')

DatePicker(value=None, description='To:')

BoundedIntText(value=0, description='Last X Days:', max=2501)

Output()

Output()

Output()

Output()